# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

    '''8053 rows'''

8056
['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88']


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### FIRST QUERY

For table song_by_sess used for first query, column sessionID was chosen as a partition key because queries will filter by it, column item_no was used as a clustering column as it's part of the filtering and provides a unique primary key for each row (song) in the session. choice of item_no as a clustering column instead of composite key of sessionID and item_no was preferred as it provides ordering as well as option to query by sessionID only for all songs in session for future use in case

In [8]:
# TABLE CREATION
query = "CREATE TABLE IF NOT EXISTS song_by_sess (sessionID int, item_no int, \
        artist text, length decimal,  \
        song text, userID int, \
        PRIMARY KEY(sessionID, item_no))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    
    

In [9]:
# Insertion from csv file into table
file = 'event_datafile_new.csv'

query = "INSERT INTO song_by_sess (sessionID, item_no, artist, length, song, userID)"
query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), \
                                line[9], int(line[10])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = "SELECT artist, song, length FROM song_by_sess WHERE sessionID= 338 AND item_no= 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)


for row in rows:
    print (row.artist, '-', row.song, '-', row.length)

Faithless - Music Matters (Mark Knight Dub) - 495.3073


### SECOND QUERY

For table song_by_use used for second query, columns sessionID and userID was chosen as a composite partition key because they are used in queries filtering. column item_no was used as a clustering column as it provides a unique primary key for each song in session as well as ordering for songs in session 

In [11]:
query = "CREATE TABLE IF NOT EXISTS song_by_use (\
        sessionID int, userID int, item_no int, \
        artist text, first_name text,\
        last_name text, song text, \
        PRIMARY KEY((sessionID, userID), item_no))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    
                    

In [12]:
file = 'event_datafile_new.csv'

query = "INSERT INTO song_by_use (sessionID, userID, item_no, artist, first_name, \
        last_name, song)"
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[1], line[4], line[9]))

In [13]:
query = "SELECT artist, song, first_name, last_name FROM song_by_use WHERE userID = 10 AND sessionID = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)


for row in rows:
    print (row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


### THIRD QUERY

For table song_by_title used for third query, column song was chosen as a partition key because queries will filter by it. column userID was used as a clustering column as it provides a unique primary key for storing each user who listened to song queried

In [14]:
query = "CREATE TABLE IF NOT EXISTS song_by_title (\
        song text, userID int, \
        first_name text, last_name text,  \
        PRIMARY KEY(song, userID))"
try:
    session.execute(query)
except Exception as e:
    print(e)                                    

In [15]:
file = 'event_datafile_new.csv'

query = "INSERT INTO song_by_title (song, userID, first_name, last_name)"
query = query + " VALUES (%s, %s, %s, %s)"
        
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (line[9], int(line[10]), line[1],  \
                                line[4]))

In [16]:
query = "SELECT first_name, last_name FROM song_by_title WHERE song= 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)


for row in rows:
    print (row.first_name, row.last_name)
   

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
session.execute('DROP TABLE IF EXISTS song_by_sess')


In [18]:
session.execute('DROP TABLE IF EXISTS song_by_use')


In [19]:
session.execute('DROP TABLE IF EXISTS song_by_title')


### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()